<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Desafio 3
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizo El Libro "Probabilidad y Estadistica para Ingenieria y Ciencias - Jay Devore - Septima Edicion" de Jay Devore

In [112]:
# Si no está instalada, hace falta instalar este modulo para leer PDFs
# pip install PyPDF2

In [2]:
# importing all the required modules
import PyPDF2

# creating a pdf reader object
reader1 = PyPDF2.PdfReader('Probabilidad y Estadistica para Ingenieria y Ciencias - Jay Devore - Septima Edicion.pdf')


texto = ""
for p in reader1.pages:
    texto+=p.extract_text()


len(texto)

2560907

In [3]:
import re
import unicodedata
import string

# El preprocesamento en castellano requiere más trabajo

# Referencia de regex:
# https://docs.python.org/3/library/re.html

def preprocess_clean_text(text):    
    # sacar tildes de las palabras
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # quitar caracteres especiales
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    text = re.sub(pattern, '', text)
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    # quitar números
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntiación
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

In [4]:
texto2 = preprocess_clean_text(texto).split('\n')

In [5]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.DataFrame(texto2, columns=['doc'])
df = df[df['doc']!=' ']
df[60:100]

,doc
60,Introduccion
61,Variables aleatorias conjuntamente distribuidas
62,Valores esperados covarianza y correlacion E...
63,Ejercicios suplementarios Bibliografia
64,Introduccion
65,Propiedades basicas de los intervalos de conf...
66,Intervalos de confianza de muestra grande par...
67,y proporcion de poblacion Introduccion
68,Algunos conceptos generales de estimacion pun...
69,Metodos de estimacion puntual


In [6]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 31424


### 1 - Preprocesamiento

In [7]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))    

In [8]:
# Demos un vistazo
sentence_tokens[:2]

[['devorepdf', 'anderixxviiiqxd', 'am', 'page', 'ivseptima', 'edicion'],
 ['probabilidad', 'y', 'estadistica']]

### 2 - Crear los vectores (word2vec)

In [9]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [10]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [11]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [12]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 31424


In [13]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 4690


### 3 - Entrenar el modelo generador

In [14]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 2082966.125
Loss after epoch 1: 1434431.625
Loss after epoch 2: 1339833.25
Loss after epoch 3: 1272517.5
Loss after epoch 4: 1247892.0
Loss after epoch 5: 1219403.5
Loss after epoch 6: 1157575.0
Loss after epoch 7: 1142637.0
Loss after epoch 8: 1127761.0
Loss after epoch 9: 1114158.0
Loss after epoch 10: 1109160.0
Loss after epoch 11: 1099924.0
Loss after epoch 12: 1087860.0
Loss after epoch 13: 1041541.0
Loss after epoch 14: 1020328.0
Loss after epoch 15: 1014472.0
Loss after epoch 16: 1011534.0
Loss after epoch 17: 1008986.0
Loss after epoch 18: 1009436.0
Loss after epoch 19: 1008438.0


(4021839, 6546420)

### 4 - Ensayar

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["probabilidad"], topn=10)

[('bilidad', 0.660105288028717),
 ('probabilidadde', 0.630105197429657),
 ('deprobabilidad', 0.6283737421035767),
 ('proba', 0.6048436760902405),
 ('dad', 0.6005218625068665),
 ('pareto', 0.5954572558403015),
 ('probabili', 0.5848279595375061),
 ('distribucionacumulativa', 0.5771308541297913),
 ('laprobabilidad', 0.5708844661712646),
 ('probabi', 0.56487637758255)]

In [16]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["error"], topn=10)

[('estan', 0.05568511039018631),
 ('hnzh', 0.04433494433760643),
 ('hhhhhs', 0.03665276616811752),
 ('ihhj', 0.03233702853322029),
 ('u', 0.028221577405929565),
 ('ejercicios', 0.013404791243374348),
 ('hhab', 0.013244781643152237),
 ('expuestos', 0.004860153421759605),
 ('ihhji', 0.004822547547519207),
 ('larga', 0.001856897841207683)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["normal"], topn=10)

[('nor', 0.6441153287887573),
 ('lognormal', 0.6213048696517944),
 ('pareto', 0.6021997928619385),
 ('distribucionde', 0.5906733274459839),
 ('beta', 0.5815991759300232),
 ('cauchy', 0.5774745345115662),
 ('gama', 0.5768176317214966),
 ('ztiene', 0.5752827525138855),
 ('geometrica', 0.5717847943305969),
 ('rayleigh', 0.570389449596405)]

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["inferencia"], topn=5)

[('ideas', 0.7595826983451843),
 ('metodologia', 0.7492762804031372),
 ('tabulares', 0.739080548286438),
 ('pictoricos', 0.7321810126304626),
 ('disciplina', 0.7254404425621033)]

In [19]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["varianza"], topn=5)

[('mador', 0.5441498756408691),
 ('desviacionestandar', 0.5249478816986084),
 ('desviacion', 0.5238469839096069),
 ('moda', 0.5186294913291931),
 ('arianza', 0.5177518129348755)]

### 5 - Visualizar agrupación de vectores

In [20]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [21]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show()
#fig.show(renderer="colab") # esto para plotly en colab


<img src="plot.png" width="2000" align="center">

### Conclusiones

- El modelo agrupa bastante bien las palabras por su significado en este contexto:
- - Palabras como modelo, regresión, intervalo, confianza aparecen cerca semánticamente, y tiene sentido
- - Hay un tema mejorable con el preprocesamiento, se ve que hay palabras que al leerlas del pdf vienen cortadas, y eso dificulta un poco.